<a href="https://colab.research.google.com/github/dolmani38/Summary/blob/master/reinforce_based_summarization_v0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Korean Summarizer Using Multiple Discriminators**

참조 : https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms

참조 : https://github.com/williamSYSU/TextGAN-PyTorch

* 2020년12월27일 v1.0 완전히 실패...
* 2020년12월27일 오후 Generator 다시 만들고... 역시 실패 한듯... v1.2 완전히 실패...
* 문법 Discriminator를 먼저 학습하고... transfer learning을 사용해 보자.
* 그래도 역시 Generator는 다시 만들어야 할 듯.

In [2]:
DO_ALL = True # 전체 실행하면서 시간 걸리는 걸 Pass 하려면 이걸 False ...


In [3]:

if DO_ALL:
    !pip install sentence-transformers==0.3.0
    !pip install transformers==3.0.2
    !pip install wikipedia
    !pip install konlpy

     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 1.5MB 25.6MB/s 
     |████████████████████████████████| 2.9MB 50.9MB/s 
     |████████████████████████████████| 890kB 54.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.0-cp36-none-any.whl size=86754 sha256=7779ff7e58580487c09f34a59a84b35b412f2d6cc8dbe8e24e46fe054641302f
  Stored in directory: /root/.cache/pip/wheels/b8/23/85/85d6a9a6c68f0625a1ecdaad903bb0a78df058c10cf74f9de4
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=70f1c84d74def44f68e4eca7ac6d48161f4f4bed35a9573dc934b5d6018b52a1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses
     |████████████████████████████████| 778kB 12.2MB/s 
     |████████████████████████████████| 1.1MB 59.7MB/s 
     |████████████████████████████████| 3.0MB 28.8MB/s 
 

In [4]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from tensorflow.python.framework import tensor_shape
import keras.utils as ku 

# set seeds for reproducability
from tensorflow.random import set_seed
from numpy.random import seed
set_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# 학습을 위한 데이터 준비

네이버 뉴스에서 아무거나 하나 Text를 얻어옴

이것을 '요약' 목표

In [5]:
org_text = """
옛날 어느 집에 귀여운 여자 아기가 태어났어요.
아기는 무럭무럭 자라서, 예쁘고 마음씨 고운 소녀가 되었어요.
그러던 어느날, 소녀의 어머니가 병이들어 그만 세상을 떠나고 말았어요.
소녀의 아버지는 홀로 남은 소녀가 걱정되었어요.
그래서 얼마 후 새어머니를 맞이했어요.
새어머니는 소녀보다 나이가 위인 두 딸을 데리고 왔어요.
그러나 새어머니와 언니들은 성질이 고약한 심술쟁이들이었어요.
새어머니는 소녀가 자기 딸들보다 예쁘고 착한 게 못마땅했어요.
그런데 이번에는 아버지마저 돌아가셨어요.
소녀는 하녀처럼 하루 종일 쓸고, 닦고, 집안일을 도맡아 했어요.
해도 해도 끝이 없는 집안일이 힘들어 지칠때면
난롯가에 앉아서 잠시 쉬곤 했지요.
"엄마, 저애를 신데렐라라고 불러야겠어요."
"온통 재투성이잖아요. 호호호!" 두 언니는 소녀를 놀려 댔어요.
어느 날, 왕궁에서 무도회가 열렸어요.
신데렐라의 집에도 초대장이 왔어요.
새어머니는 언니들을 데리고 무도회장으로 떠났어요.
신데렐라도 무도회에 가고 싶었어요.
혼자 남은 신데렐라는 훌쩍훌쩍 울기 시작했어요.
"신데렐라, 너도 무도회에 가고 싶니?"
신데렐라가 고개를 들어보니, 마법사 할머니가 빙그레 웃고 있었어요.
"내가 너를 무도회에 보내주마 호박 한개와 생쥐 두마리, 도마뱀을 구해 오렴."
마법사 할머니가 주문을 외웠어요.
그리고 지팡이로 호박을 건드리자, 호박이 화려한 황금 마차로 변했어요.
이번에는 생쥐와 도마뱀을 건드렸어요.
그랬더니 생쥐는 흰말로, 도마뱀은 멋진 마부로 변했답니다.
신데렐라의 옷도 구슬 장식이 반짝이는 예쁜 드레스로 바뀌웠어요.
"신데렐라, 발을 내밀어 보거라."
할머니는 신데렐라에게 반짝반짝 빛나는 유리 구두를 신겨 주었어요.
"신데렐라, 밤 열두시가 되면 모든게 처음대로 돌아간단다. 황금 마차는 호박으로, 흰말은 생쥐로, 마부는 도마뱀으로 변하게 돼. 그러니까 반드시 밤 열두 시가 되기 전에 돌아와야 해. 알겠지?"
왕자님도 아름다운 신데렐라에게 마음을 빼았겼어요.
왕자님은 무도회장에 모인 다른 아가씨들은 쳐다보지도 않고,
신데렐라하고만 춤을 추었어요.
신데렐라는 왕자님과 춤을 추느라 시간 가는 줄도 몰랐어요.
땡, 땡, 땡...... 벽시계가 열두 시를 알리는 소리에 신데렐라는 화들짝 놀랐어요.
신데렐라가 허둥지둥 왕궁을 빠져나가는데,
유리 구두 한 짝이 벗겨졌어요.
하지만 구두를 주울 틈이 없었어요.
신데렐라를 뛰쫓아오던 왕자님은 층계에서 유리 구두 한 짝을 주웠어요.
왕자님은 유리 구두를 가지고 임금님께 가서 말했어요.
"이 유리 구두의 주인과 결혼하겠어요."
그래서 신하들은 유리 구두의 주인을 찾아 온 나라를 돌아다녔어요.
언니들은 발을 오므려도 보고, 구두를 늘려도 보았지만 한눈에 보기에도 유리 구두는 너무 작았어요.
그때, 신데렐라가 조용히 다가와 말했어요.
"저도 한번 신어 볼 수 있나요?"
신데렐라는 신하게 건넨 유리 구두를 신었어요,
유리 구두는 신데렐라의 발에 꼭 맞았어요.
신하들은 신데렐라를 왕궁으로 데리고 갔어요.
그 뒤 신데렐라는 왕자님과 결혼하여 오래오래 행복하게 살았대요.
"""

In [6]:
# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    return txt 

org_text = clean_text(org_text).strip()
org_text

'옛날 어느 집에 귀여운 여자 아기가 태어났어요. 아기는 무럭무럭 자라서 예쁘고 마음씨 고운 소녀가 되었어요. 그러던 어느날 소녀의 어머니가 병이들어 그만 세상을 떠나고 말았어요. 소녀의 아버지는 홀로 남은 소녀가 걱정되었어요. 그래서 얼마 후 새어머니를 맞이했어요. 새어머니는 소녀보다 나이가 위인 두 딸을 데리고 왔어요. 그러나 새어머니와 언니들은 성질이 고약한 심술쟁이들이었어요. 새어머니는 소녀가 자기 딸들보다 예쁘고 착한 게 못마땅했어요. 그런데 이번에는 아버지마저 돌아가셨어요. 소녀는 하녀처럼 하루 종일 쓸고 닦고 집안일을 도맡아 했어요. 해도 해도 끝이 없는 집안일이 힘들어 지칠때면 난롯가에 앉아서 잠시 쉬곤 했지요. 엄마 저애를 신데렐라라고 불러야겠어요. 온통 재투성이잖아요. 호호호! 두 언니는 소녀를 놀려 댔어요. 어느 날 왕궁에서 무도회가 열렸어요. 신데렐라의 집에도 초대장이 왔어요. 새어머니는 언니들을 데리고 무도회장으로 떠났어요. 신데렐라도 무도회에 가고 싶었어요. 혼자 남은 신데렐라는 훌쩍훌쩍 울기 시작했어요. 신데렐라 너도 무도회에 가고 싶니? 신데렐라가 고개를 들어보니 마법사 할머니가 빙그레 웃고 있었어요. 내가 너를 무도회에 보내주마 호박 한개와 생쥐 두마리 도마뱀을 구해 오렴. 마법사 할머니가 주문을 외웠어요. 그리고 지팡이로 호박을 건드리자 호박이 화려한 황금 마차로 변했어요. 이번에는 생쥐와 도마뱀을 건드렸어요. 그랬더니 생쥐는 흰말로 도마뱀은 멋진 마부로 변했답니다. 신데렐라의 옷도 구슬 장식이 반짝이는 예쁜 드레스로 바뀌웠어요. 신데렐라 발을 내밀어 보거라. 할머니는 신데렐라에게 반짝반짝 빛나는 유리 구두를 신겨 주었어요. 신데렐라 밤 열두시가 되면 모든게 처음대로 돌아간단다. 황금 마차는 호박으로 흰말은 생쥐로 마부는 도마뱀으로 변하게 돼. 그러니까 반드시 밤 열두 시가 되기 전에 돌아와야 해. 알겠지? 왕자님도 아름다운 신데렐라에게 마음을 빼았겼어요. 왕자님은 무도회장에 모인 다른 아가씨들은 쳐다보지도 않고 신데렐라하고만 

## 한국어 문법 구분 discriminator 학습

* '한글 위키백과'에서 임의의 문장을 수집 함

In [6]:
#한국어 위키백과에서 스크랩핑

import wikipedia as wiki
wiki.set_lang('ko')

In [40]:
# '전래동화' 라는 keyword로 100개 page의 Text를 취득

def __search_from_wiki(question,max_rank):
    results = wiki.search(question,results=max_rank)
    print(results)
    contents = []
    for result in results:
        try:
            page = wiki.page(result)
            #print(f"Top wiki result: {page}")
            text = page.content
            ln = len(text)
            #print(f'Collecting page : {page} , text length {str(ln)}')
            #if ln < 4000:
            #  contents.append(text)
            #else:
            #  contents.append(text[0:4000])
            contents.append(text)
        except Exception as ex:
            print(ex)
    return contents

if DO_ALL:
    ko_grammar_set_raw = __search_from_wiki("동화",300)
    #ko_grammar_set_raw = __search_from_wiki("소설",300)
    #ko_grammar_set_raw = __search_from_wiki("사설",300)
    #ko_grammar_set_raw = __search_from_wiki("설명",300)
    print(f'전체 수집한 Page Count : {len(ko_grammar_set_raw)}')


['동화', '니코니코 동화', '동화중학교 (경기)', '문화 동화', '조곡 니코니코 동화', '동화사', '신에이 동화', '동화작용', '번암초등학교 동화분교', '책상 서랍 속의 동화', '동화초등학교 (경기)', '동화약품', '동화기업', '동화면세점', '동화역', '동화고등학교', '동화초등학교 (전남)', '화성동화중학교', '그림 동화', '동화초등학교 (강원)', '단백동화 스테로이드', '가을동화', '동화 (언어학)', '애니메이션', '동화초등학교 (충북)', '수성알파시티 동화아이위시', '동화도서관', '훈1등 욱일동화대수장', '동화초등학교', '도에이 애니메이션', '동화중학교 (강원)', '동화은행', '대전동화초등학교', '전북동화중학교', 'TV로 보는 원작동화', '동윤', '신 전래동화', '대전동화중학교', '동화초등학교 (제주)', '한스 크리스티안 안데르센', '도라에몽 (1973년 애니메이션)', '대원씨아이', '빌헬름 하우프', '동화면', '그림 형제', '동화 읽어주는 TV', '빛가람장성로', '아동 문학', 'TV동화 행복한 세상', '권설 설측 접근음', '촉한', '정이서', '동화운수 (인천)', '질소 고정', '대구막항', '동문항', '대구 동화사 비로암 삼층석탑', '전한', '대한민국의 국회의원 선거구 목록', '유글레나류', '리하르트 빌슈테터', '코파반장의 동화수사대', '쥐', '팔공산', '리루프릿', '스튜디오 지브리', '1844년', '동화 (후한)', '동화처럼', '동화운수 (광주)', '대구 동화사 당간지주', '주엽어린이도서관', '아돌프 폰 바이어', '必UP되다', '신-맨', '산서초등학교 (전북)', '대구 동화사 마애여래좌상', '샤를 페로', '등장인물', '동화 (화폐)', '오엘엠', '삼성물산', '란티스 조곡 feat.Nico Nico Artists', '스튜디오 딘', '루트비히 티크', '일본동화협회', '한국의 사찰', '곰돌이 푸', '김지

In [41]:
# 간단한 전처리

ko_grammar_set_raw = [clean_text(x) for x in ko_grammar_set_raw]
print('Sample text : ')
print('--------------------------------------------------------------------------------------------')
print(ko_grammar_set_raw[296])
print('--------------------------------------------------------------------------------------------')

Sample text : 
--------------------------------------------------------------------------------------------
베이후 구(한국어: 북호구 중국어: 北湖区 병음: Běihú Qū)는 중화 인민 공화국 후난성 천저우 시의 현급 행정구역이다. 넓이는 815km2이고 인구는 2007년 기준으로 330000명이다.   행정 구역 4개 가도 5개 진 7개 향 2개 민족향을 관할한다.  가도: 人民路街道 北湖街道 燕泉街道 下湄桥街道. 진: 石盖塘镇 华塘镇 鲁塘镇 郴江镇 万华岩镇. 향: 江口乡 市郊乡 同和乡 保和乡 芙蓉乡 永春乡 南溪乡. 민족향: 大塘瑶族乡 月峰瑶族乡. 
--------------------------------------------------------------------------------------------


In [42]:
# 긴 소설하나 줍줍

filepath = 'korean_sample.txt'
korean_sample = ""
file1 = open(filepath, 'r') 
Lines = file1.readlines()
for line in Lines:
    line = line.strip()
    if line.startswith(('\"','\'',' ')) or line.endswith(('\"','\'')) or line=='':
        pass
    else:
        if len(line) > 10:
            #print(line)
            korean_sample += line

korean_sample = clean_text(korean_sample)


In [43]:
ko_grammar_set_raw += [korean_sample]
print(f'전체 수집한 Page Count : {len(ko_grammar_set_raw)}')

전체 수집한 Page Count : 298


문장으로 잘라 낸다

In [44]:
import nltk
nltk.download('punkt')
#Split the document into sentences
ko_grammar_sentences = []
for document in ko_grammar_set_raw:
    ko_grammar_sentences += nltk.sent_tokenize(document)

print("Num sentences:", len(ko_grammar_sentences))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Num sentences: 32705


In [48]:
ko_grammar_sentences[30010]

'공간이동을 한두 번 겪은 것도 아니었지만 틸라크에서 헤모시아를 거쳐 다시 아라사로 이동하는 수천 파르상이나 되는 장거리 공간이동은 처음이었기에 현기증을 느끼다못해 구토까지 일 것 같았다.'

In [49]:
# false 문장을 만들기 위해 shffle 함수 준비
import random

def shuffling(txt):
    txt_list = txt.split(' ')
    random.shuffle(txt_list)
    return ' '.join(txt_list)

# true 문장, false 문장의 생성
ko_grammar_dataset = []
for txt in ko_grammar_sentences:
    ko_grammar_dataset.append([txt,1])
    ko_grammar_dataset.append([shuffling(txt),0])
    
# dataset을 전체적으로 다시 썩는다.
random.shuffle(ko_grammar_dataset)

형태소 분리하여 모든 문장을 형태소 Code로 변환 한다.

In [52]:
from konlpy.tag import Twitter
twitter = Twitter()

# 형태소 Code table의 구성

_MAX_MORP_LENGTH = 128
_PADDING_CODE = 0  # padding code
_MISMATCH_CODE = 1 # mismatch word code               ex) @@@
_MISMATCH_WORD = '@@@' # 이거 아래에서 쓴다.

morpheme_table = {}
morp_code = _MISMATCH_CODE+1
morpheme_table['Pad'] = _PADDING_CODE 
morpheme_table['Mst'] = _MISMATCH_CODE 
for sentence in ko_grammar_sentences[:7000]:
    morphemes = twitter.pos(sentence)
    for (word,morp) in morphemes:
        if morp in morpheme_table:
            pass
        else:
            morpheme_table[morp] = morp_code
            morp_code += 1


In [54]:
morpheme_table['Hashtag']=20 # 빠진거 채워넣음...
morpheme_table['URL']=21 # 빠진거 채워넣음...

In [55]:
print('Korean morpheme code table')
print('----------------------------------------------------------')
print('  Morpheme        Code')
print('')
for morp in morpheme_table.keys():
    print(f' {morp.ljust(15)}   {morpheme_table[morp]}')
print('----------------------------------------------------------')


Korean morpheme code table
----------------------------------------------------------
  Morpheme        Code

 Pad               0
 Mst               1
 Noun              2
 Punctuation       3
 Foreign           4
 Josa              5
 Verb              6
 Modifier          7
 Adjective         8
 Suffix            9
 Adverb            10
 Number            11
 Alpha             12
 Determiner        13
 Conjunction       14
 Exclamation       15
 KoreanParticle    16
 VerbPrefix        17
 Eomi              18
 PreEomi           19
 Hashtag           20
 URL               21
----------------------------------------------------------


In [56]:
# morpheme 코드 변환기 준비
def morpheme_encode(sentence):
    encode=[]
    morphemes = twitter.pos(sentence)
    for (word,morp) in morphemes:
        encode.append(_MISMATCH_CODE if word==_MISMATCH_WORD else morpheme_table[morp])

    if len(encode) <= _MAX_MORP_LENGTH:
        encode = encode + [_PADDING_CODE for i in range(_MAX_MORP_LENGTH-len(encode))]
    else:
        encode = encode[:_MAX_MORP_LENGTH]       
    return encode

#true / false 문장을 morpheme 코드로 모두 변환
ko_morpheme_x = []
ko_morpheme_y = []
for (txt,label) in ko_grammar_dataset:
    ko_morpheme_x.append(morpheme_encode(txt))
    ko_morpheme_y.append([label])

ko_morpheme_x = np.asarray(ko_morpheme_x)
ko_morpheme_y = np.asarray(ko_morpheme_y)  

trainset 과 testset의 분리 

In [57]:
from sklearn.model_selection import train_test_split
# 20%를 testset으로 사용.,,
X_train, X_test, y_train, y_test = train_test_split(ko_morpheme_x,ko_morpheme_y,test_size=0.2)

print(f'Shape of X_train;{X_train.shape}')
print(f'Shape of X_test ;{X_test.shape}')
print(f'Shape of y_train;{y_train.shape}')
print(f'Shape of y_test ;{y_test.shape}')

Shape of X_train;(52328, 128)
Shape of X_test ;(13082, 128)
Shape of y_train;(52328, 1)
Shape of y_test ;(13082, 1)


Model을 생성하고 학습 시킨다.

In [59]:
# model build

morpheme_model = Sequential()
morpheme_model.add(Dense(500, input_dim=128, activation= "relu"))
morpheme_model.add(Dense(1000, activation= "relu"))
morpheme_model.add(Dense(100, activation= "relu"))
morpheme_model.add(Dense(50, activation= "relu"))
morpheme_model.add(Dense(1, activation= "sigmoid"))
morpheme_model.summary() #Print model Summary


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 500)               64500     
_________________________________________________________________
dense_6 (Dense)              (None, 1000)              501000    
_________________________________________________________________
dense_7 (Dense)              (None, 100)               100100    
_________________________________________________________________
dense_8 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 51        
Total params: 670,701
Trainable params: 670,701
Non-trainable params: 0
_________________________________________________________________


In [60]:
morpheme_model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
morpheme_model.fit(X_train, y_train, epochs=100)

Epoch 1/100
1636/1636 [==============================] - 6s 2ms/step - loss: 0.2487 - mean_squared_error: 0.2487
Epoch 2/100
1636/1636 [==============================] - 4s 2ms/step - loss: 0.2111 - mean_squared_error: 0.2111
Epoch 3/100
1636/1636 [==============================] - 3s 2ms/step - loss: 0.1741 - mean_squared_error: 0.1741
Epoch 4/100
1636/1636 [==============================] - 3s 2ms/step - loss: 0.1593 - mean_squared_error: 0.1593
Epoch 5/100
1636/1636 [==============================] - 3s 2ms/step - loss: 0.1482 - mean_squared_error: 0.1482
Epoch 6/100
1636/1636 [==============================] - 3s 2ms/step - loss: 0.1420 - mean_squared_error: 0.1420
Epoch 7/100
1636/1636 [==============================] - 4s 2ms/step - loss: 0.1371 - mean_squared_error: 0.1371
Epoch 8/100
1636/1636 [==============================] - 4s 2ms/step - loss: 0.1293 - mean_squared_error: 0.1293
Epoch 9/100
1636/1636 [==============================] - 4s 2ms/step - loss: 0.1262 - mean_squar

In [61]:
# 학습결과 확인

results = morpheme_model.evaluate(X_test, y_test)
print(results)

409/409 [==============================] - 1s 2ms/step - loss: 0.1686 - mean_squared_error: 0.1686
[0.1685619205236435, 0.1685619205236435]


In [62]:
# 학습 결과의 확인
predicts = morpheme_model.predict(X_test)
predicts = np.asarray(predicts)
predicts = [ 1 if x>0.5 else 0 for [x] in predicts]
y = np.asarray(y_test)
y = y.reshape(y.shape[0],)

from sklearn.metrics import classification_report
print(classification_report(y, predicts))

              precision    recall  f1-score   support

           0       0.84      0.75      0.79      6556
           1       0.77      0.85      0.81      6526

    accuracy                           0.80     13082
   macro avg       0.81      0.80      0.80     13082
weighted avg       0.81      0.80      0.80     13082



In [63]:
# 모델의 저장

morpheme_model.save('morpheme_model.h5')
morpheme_model.save_weights('morpheme_model.weights')

In [64]:
def morpheme_discriminator(queries):
    # queries : 복수의 문장의 2차원 배열 (None,1)
    # return : 결과 score 배열 (None,)
    x_codes = []
    for query in queries:
        x_codes.append(morpheme_encode(query))
    scores = morpheme_model.predict(x_codes)
    return scores

In [65]:
ko_grammar_sentences[100:102]

['바로 옆에 동화고등학교가 있다.', '건물은 사랑관 소망관 믿음관 송학관 여호수아홀이 있다.']

In [66]:
morpheme_discriminator(ko_grammar_sentences[100:102])

array([[0.9388542 ],
       [0.97550076]], dtype=float32)

## 문서 유사도 Discriminator 준비

In [7]:
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer

if DO_ALL:
    # embedder download...
    embedder = SentenceTransformer('xlm-r-large-en-ko-nli-ststb')

100%|██████████| 1.80G/1.80G [01:14<00:00, 24.3MB/s]


In [8]:

# 생성된 문장의 원문 유사도를 측정하기 위한 함수

import scipy
#print(doc_emb)
def similarity_discriminator(queries,org_embedding):
    # queries : 복수의 문장의 2차원 배열 (None,1)
    # org_embedding : 비교 대상의 원문 embedding 1차원 배열 (1,)
    # return : 결과 score 배열 (None,)
    total_score = 0
    query_embeddings = embedder.encode(queries,show_progress_bar=False)
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], [org_embedding], "cosine")[0]
        results = zip(range(len(distances)), distances)
        for idx, distance in results:
            total_score += 1-distance
    return total_score

In [9]:
# 원문의 embedding...
org_text_emb = embedder.encode([org_text],show_progress_bar=False)[0]

In [10]:
org_text_emb

array([ 0.15200967,  0.92120737, -0.1766923 , ..., -0.72806704,
        0.742245  , -0.41552904], dtype=float32)

## 원문 base의 generator 준비 

In [11]:
org_term_set = org_text.split(' ')

_MAX_GEN_TOKEN = 40
_NOISE_DIM = len(org_term_set)

word_table = {}

for index, word in zip(range(len(org_term_set)),org_term_set):
    word_table[index] = word

In [77]:
print('Token table of origin text')
print('---------------------------------------------')
print(' Code         Token      ')
print('')
for k in word_table.keys():
  print( f'  {str(k).ljust(8)}    {word_table[k]}')
print('---------------------------------------------')

Token table of origin text
---------------------------------------------
 Code         Token      

  0           옛날
  1           어느
  2           집에
  3           귀여운
  4           여자
  5           아기가
  6           태어났어요.
  7           아기는
  8           무럭무럭
  9           자라서
  10          예쁘고
  11          마음씨
  12          고운
  13          소녀가
  14          되었어요.
  15          그러던
  16          어느날
  17          소녀의
  18          어머니가
  19          병이들어
  20          그만
  21          세상을
  22          떠나고
  23          말았어요.
  24          소녀의
  25          아버지는
  26          홀로
  27          남은
  28          소녀가
  29          걱정되었어요.
  30          그래서
  31          얼마
  32          후
  33          새어머니를
  34          맞이했어요.
  35          새어머니는
  36          소녀보다
  37          나이가
  38          위인
  39          두
  40          딸을
  41          데리고
  42          왔어요.
  43          그러나
  44          새어머니와
  45          언니들은
  46          성질이
  47          고약한
  48          심술쟁이들이었어요.

In [ ]:
_MAX_GEN_TOKEN = 40



In [36]:
tc = 5
a = [1,2,3,4,5,6,7,8,5,10]
s_a = np.sort(a)[::-1]
print(s_a)

o = np.where(a > s_a[tc+1])[0][-tc:]
print('order:',o)
print('value:',np.asarray(a)[o])

print(len(o))

[10  8  7  6  5  5  4  3  2  1]
order: [5 6 7 8 9]
value: [ 6  7  8  5 10]
5


In [ ]:
noise = np.random.rand(_NOISE_DIM,)
def text_gen(noise):
    text = ""
    sorted_noise = np.sort(noise)[::-1]
    order = np.where(noise > sorted_noise[_MAX_GEN_TOKEN+1])[0][-_MAX_GEN_TOKEN:]
    assert len(order) == _MAX_GEN_TOKEN
    for k in order:
        text += word_table[k]+' '    
    return text,order 

text,order = text_gen(noise)

text

In [ ]:
'''
for i in range(10):
    noise = tf.random.normal([1,_NOISE_DIM])
    texts,embeddings,compratios,morpcodes = generator(noise, training=True)
    p_score = 0
    for txt in texts:
        embedding = embedder.encode([txt.numpy().decode('utf-8')],show_progress_bar=False)
        distances = scipy.spatial.distance.cdist(embedding, [org_text_emb], "cosine")[0]
        score = 1-distances[0]
        reward = p_score - score
        if p_score == 0:
            reward = -reward
        print(f'score:{score} reward:{reward}') # 1-distance
        p_score = score
'''